In [1]:
import numpy as np
import pandas as pd
import keras
import os
from pathlib import Path

In [2]:
#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

In [3]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [4]:
batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [5]:
data_path = "fra.txt"

In [6]:
input_texts = []
target_texts = []
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = target_text
    input_texts.append(input_text)
    target_texts.append(target_text)

In [7]:
exclude = set(string.punctuation)

In [8]:
remove_digits = str.maketrans('', '', digits)

In [9]:
input_data = []
for i in range(0, len(input_texts)):
    review = input_texts[i]
    review = review.lower()
    review = re.sub("'", '', review)
    review = re.sub(",", ' COMMA', review) 
    review = [ch for ch in review if ch not in exclude]
    review = ''.join(review)
    review = review.translate(remove_digits)
    input_data.append(review)

In [10]:
target_data = []
for i in range(0, len(target_texts)):
    review = target_texts[i]
    review = review.lower()
    review = re.sub("'", '', review)
    review = re.sub(",", ' COMMA', review)
    review = [ch for ch in review if ch not in exclude]
    review = ''.join(review)
    review = review.translate(remove_digits)
    review = 'START_ '+ review + ' _END'
    target_data.append(review)

In [11]:
target_data[0]

'START_ va  _END'

In [12]:
english_words=set()
for eng in input_data:
    for word in eng.split():
        if word not in english_words:
            english_words.add(word)

In [13]:
french_words=set()
for french in target_data:
    for word in french.split():
        if word not in french_words:
            french_words.add(word)

In [14]:
print(len(english_words))
print(len(french_words))

2016
4411


In [15]:
lenght_list_english=[]
for l in input_data:
    lenght_list_english.append(len(l.split(' ')))
np.max(lenght_list_english)

5

In [16]:
lenght_list_french=[]
for l in target_data:
    lenght_list_french.append(len(l.split(' ')))
np.max(lenght_list_french)

12

In [17]:
all_english_words = sorted(list(english_words))
all_french_words = sorted(list(french_words))
english_vocab_size = len(all_english_words)
french_vocab_size = len(all_french_words)
english_max_sent_length = np.max(lenght_list_english)
french_max_sent_length = np.max(lenght_list_french)

In [18]:
english_token_index = dict([(word, i) for i, word in enumerate(all_english_words)])
french_token_index = dict([(word, i) for i, word in enumerate(all_french_words)])

In [19]:
english_max_sent_length = french_max_sent_length

In [20]:
encoder_input_data = np.zeros((num_samples, english_max_sent_length), dtype='float32')
decoder_input_data = np.zeros((num_samples, french_max_sent_length), dtype='float32')
decoder_target_data = np.zeros((num_samples, french_max_sent_length, french_vocab_size), dtype='float32')

In [21]:
for i, (input_text, target_text) in enumerate(zip(input_data, target_data)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = english_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = french_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1,french_token_index[word]] = 1.

In [22]:
embedding_size = 256
latent_dim = 256

In [23]:
from keras.layers import Input, LSTM, Embedding, Dense, Attention, Concatenate, TimeDistributed
from keras.models import Model

In [24]:
#encoder model
encoder_inputs = Input(shape=(12,))
en_x=  Embedding(english_vocab_size, embedding_size)(encoder_inputs)
encoder_lstm_1 = LSTM(latent_dim,return_state=True,return_sequences=True)
encoder_output1, state_h1, state_c1  =  encoder_lstm_1(en_x)

#encoder_lstm_2 = LSTM(latent_dim, return_state=True,return_sequences=True)
#encoder_output2, state_h2, state_c2 = encoder_lstm_2(encoder_output1)

In [25]:
encoder_output1.shape

TensorShape([None, 12, 256])

In [26]:
#decoder model
decoder_inputs = Input(shape=(None,))
dex=  Embedding(french_vocab_size, embedding_size)
final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state=[state_h1, state_c1])

In [27]:
#attention layer
attention = Attention()
attn_out=attention([encoder_output1,decoder_outputs])

In [28]:
#combine attention with decoder outputs
decoder_outputs = Concatenate(axis=-1)([ decoder_outputs,attn_out])
decoder_dense = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
decoder_final_outputs=decoder_dense(decoder_outputs)
seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_final_outputs)
seq2seq_Model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])

In [29]:
seq2seq_Model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size,
          epochs,
          validation_split=0.05)

Epoch 1/5
149/149 [==============================] - 88s 563ms/step - loss: 1.6579 - accuracy: 0.1035 - val_loss: 1.6989 - val_accuracy: 0.1238
Epoch 2/5
149/149 [==============================] - 78s 524ms/step - loss: 1.2850 - accuracy: 0.1396 - val_loss: 1.3860 - val_accuracy: 0.1868
Epoch 3/5
149/149 [==============================] - 79s 529ms/step - loss: 1.0739 - accuracy: 0.1674 - val_loss: 1.2425 - val_accuracy: 0.2087
Epoch 4/5
149/149 [==============================] - 78s 522ms/step - loss: 0.9391 - accuracy: 0.1856 - val_loss: 1.1454 - val_accuracy: 0.2212
Epoch 5/5
149/149 [==============================] - 77s 517ms/step - loss: 0.8384 - accuracy: 0.1998 - val_loss: 1.0936 - val_accuracy: 0.2353
